In [4]:
!pip install tensorflow==2.16.1 opencv-python-headless pillow lxml matplotlib --quiet
!git clone https://github.com/tensorflow/models.git
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!pip install .
%cd ../..

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.16.1 which is incompatible.
tf-models-official 2.19.1 requires tensorflow~=2.19.0, but you have tensorflow 2.16.1 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.16.1 which is incompatible.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.3.2 which is incompatible.
fatal: destination path 'models' already exists and is not an empty directory.
/content/models/research
Processing /content/models/research
  Preparing metadata (setup.py) ... done
  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  U

In [7]:
from google.colab import files
uploaded = files.upload()

Saving archive.zip to archive.zip


In [8]:
import zipfile
import os

zip_path = "archive.zip"  # same name you uploaded
extract_dir = "dataset"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("✅ Dataset extracted to:", extract_dir)

✅ Dataset extracted to: dataset


In [9]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
base_dir = 'dataset/archive'  # because zip extracted into this path

In [14]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3561 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set base directory — update this if your folder name is different
base_dir = 'dataset/archive'  # this is where Butterfly, Mosquito etc. folders are

# Create ImageDataGenerator for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)

# Training data generator
train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation data generator
validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 3561 images belonging to 5 classes.
Found 888 images belonging to 5 classes.


In [14]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=1  # You can increase for better accuracy
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


112/112 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2881 - loss: 2.4831

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


112/112 ━━━━━━━━━━━━━━━━━━━━ 174s 2s/step - accuracy: 0.2887 - loss: 2.4760 - val_accuracy: 0.3637 - val_loss: 1.4864


In [15]:
# Save the model even if partially trained
model.save("quick_model.h5")

In [ ]:
import matplotlib.pyplot as plt

# Accuracy
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.legend()
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.show()

# Loss
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [17]:
val_loss, val_acc = model.evaluate(validation_generator)
print(f"Validation Accuracy: {val_acc*2:.2f}%")

28/28 ━━━━━━━━━━━━━━━━━━━━ 16s 529ms/step - accuracy: 0.4013 - loss: 1.4485
Validation Accuracy: 38.51%


In [20]:
!ls dataset/archive/Butterfly/

google0.jpg    google341.jpg  google573.jpg	  istockphoto308.jpg
google100.jpg  google342.jpg  google574.jpg	  istockphoto30.jpg
google101.jpg  google343.jpg  google575.jpg	  istockphoto310.jpg
google102.jpg  google344.jpg  google576.jpg	  istockphoto311.jpg
google103.jpg  google345.jpg  google578.jpg	  istockphoto312.jpg
google104.jpg  google346.jpg  google579.jpg	  istockphoto313.jpg
google105.jpg  google347.jpg  google57.jpg	  istockphoto314.jpg
google107.jpg  google349.jpg  google580.jpg	  istockphoto315.jpg
google109.jpg  google34.jpg   google581.jpg	  istockphoto316.jpg
google10.jpg   google350.jpg  google582.jpg	  istockphoto318.jpg
google110.jpg  google351.jpg  google583.jpg	  istockphoto31.jpg
google111.jpg  google352.jpg  google584.jpg	  istockphoto32.jpg
google112.jpg  google353.jpg  google586.jpg	  istockphoto342.jpg
google113.jpg  google354.jpg  google587.jpg	  istockphoto343.jpg
google114.jpg  google355.jpg  google588.jpg	  istockphoto344.jpg
google118.jpg  google356.jpg 

In [23]:
import numpy as np
from tensorflow.keras.preprocessing import image

img_path = "dataset/archive/Butterfly/google0.jpg"  # Updated to a valid filename
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

prediction = model.predict(img_array)
predicted_class = train_generator.class_indices
predicted_class = list(predicted_class.keys())[np.argmax(prediction)]

print("Predicted class:", predicted_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
Predicted class: Dragonfly


In [24]:
model.save("insect_classifier_model.h5")